In [17]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

massa1 = float(input("Massa 1: "))
massa2 = float(input("Massa 2: "))
panjang1 = float(input("Panjang Tali 1: "))
panjang2 = float(input("Panjang Tali 2: "))
sudut1 = float(input("Sudut 1: "))
sudut2 = float(input("Sudut 2: "))
time = float(input("Waktu Simulasi: "))
dt = float(input("Step Size: "))

sudut = np.array([[sudut1*np.pi/180], [sudut2*np.pi/180]])
sudutdot = np.array([[0], [0]])
tau = np.array([[0], [0]])
i1 = massa1*(panjang1**2)
i2 = massa2*(panjang2**2)


def sudutkeXY():
    global panjang1, panjang2, sudut
    garisXY = np.array([[0, 0], [panjang1*np.sin(sudut[0, 0]), (-panjang1*np.cos(sudut[0, 0]))], [panjang1*np.sin(
        sudut[0, 0])+panjang2*np.sin(sudut[1, 0]), -(panjang1*np.cos(sudut[0, 0])+panjang2*np.cos(sudut[1, 0]))]])
    massaXY = np.array([[panjang1*np.sin(sudut[0, 0])/2, -panjang1*np.cos(sudut[0, 0])/2], [panjang1*np.sin(
        sudut[0, 0])+panjang2*np.sin(sudut[1, 0])/2, -(panjang1*np.cos(sudut[0, 0])+1/2*panjang2*np.cos(sudut[1, 0]))]])
    return garisXY, massaXY


def inv(matrix):
    return np.array([[matrix[1, 1], -1*matrix[0, 1]], [-1*matrix[1, 0], matrix[0, 0]]])/matrix[0, 0]*matrix[1, 1]-matrix[0, 1]*matrix[1, 0]


def pendulum_lagrangian(sudut, sudutdot):
    global massa1, massa2, panjang1, panjang2, i1, i2, tau
    g = 9.8
    M = np.array([[(massa1/4+massa2)*(panjang1**2)+i1, massa2*panjang1*panjang2*np.cos(sudut[0, 0]-sudut[1, 0])/2],
                 [massa2*panjang1*panjang2*np.cos(sudut[0, 0]-sudut[1, 0])/2, massa2*(panjang2**2)/4+i2]])
    Minv = inv(M)
    C = np.array([[massa2*panjang1*panjang2*np.sin(sudut[0, 0]-sudut[1, 0])*sudutdot[1, 0]/2, -1*massa2*panjang1*panjang2*np.sin(sudut[0, 0]-sudut[1, 0])*(sudutdot[0, 0]-sudutdot[1, 0])/2],
                 [-1*massa2*panjang1*panjang2*np.sin(sudut[0, 0]-sudut[1, 0])*(sudutdot[0, 0]-sudutdot[1, 0])/2, -1*massa2*panjang1*panjang2*np.sin(sudut[0, 0]-sudut[1, 0])*sudutdot[0, 0]/2]])
    A = np.matmul(Minv, C)
    G = np.array([[((massa1/2)+massa2)*g*panjang1*np.sin(sudut[0, 0])],
                  [massa2*g*panjang2*np.sin(sudut[1, 0])/2]])
    return np.matmul(Minv, tau)-np.matmul(A, sudutdot)-np.matmul(Minv, G)


def rungekutta4():
    global sudut, sudutdot,  dt
    k1 = dt/2*pendulum_lagrangian(sudut, sudutdot)
    k2 = dt/2*pendulum_lagrangian(sudut+(dt/2)*(sudutdot+(k1/2)), sudutdot+k1)
    k3 = dt/2*pendulum_lagrangian(sudut+(dt/2)*(sudutdot+k1/2), sudutdot+k2)
    k4 = dt/2*pendulum_lagrangian(sudut+dt*(sudutdot+k3), sudutdot+2*k3)
    sudut = sudut + dt*(sudutdot+((k1+k2+k3)/3))
    sudutdot = sudutdot + (k1+(2*k2)+(2*k3)+k4)/3


%matplotlib

fig = plt.figure(figsize=(10, 20), constrained_layout=True)
gs = fig.add_gridspec(6, 8)

ax_pendulum_lagrangian = fig.add_subplot(gs[:, :4])
ax_sudut1 = fig.add_subplot(gs[0:2, 4:])
ax_sudut2 = fig.add_subplot(gs[2:4, 4:])
ax_sudutdot1 = fig.add_subplot(gs[4:6, 4:6])
ax_sudutdot2 = fig.add_subplot(gs[4:6, 6:8])

ax_pendulum_lagrangian.set_title('Double Pendulum')
ax_sudut1.set_title('Sudut 1')
ax_sudut2.set_title('Sudut 2')
ax_sudutdot1.set_title('Turunan Sudut 1 terhadap Sudut 1')
ax_sudutdot2.set_title('Turunan Sudut 2 terhadap Sudut 2')

ax_sudut1.set_xlabel("s")
ax_sudut2.set_xlabel("s")
ax_sudut1.set_ylabel("Radian")
ax_sudut2.set_ylabel("Radian")

garisXY, massaXY = sudutkeXY()

plot_massa, = ax_pendulum_lagrangian.plot(
    massaXY[:, 0], massaXY[:, 1], 'ro', zorder=4)
plot_garis, = ax_pendulum_lagrangian.plot(
    garisXY[:, 0], garisXY[:, 1], 'r-', zorder=2)
plot_jejak_massa, = ax_pendulum_lagrangian.plot([], [], 'bo', zorder=0)

plot_sudut1, = ax_sudut1.plot([], [], '-')
plot_sudut2, = ax_sudut2.plot([], [], '-')
plot_sudutdot1, = ax_sudutdot1.plot([], [], '-')
plot_sudutdot2, = ax_sudutdot2.plot([], [], '-')

list_jejak_sudut1 = [sudut[0, 0]]
list_jejak_sudut2 = [sudut[1, 0]]
list_jejak_sudutdot1 = [sudutdot[0, 0]]
list_jejak_sudutdot2 = [sudutdot[1, 0]]
list_jejak = np.array([massaXY[0, :], massaXY[1, :]])


def inisialisasi_animasi():
    global panjang1, panjang2, time
    ax_pendulum_lagrangian.set_xlim(-(panjang1 +
                                    panjang2+0.1), (panjang1+panjang2+0.1))
    ax_pendulum_lagrangian.set_ylim(-(panjang1 +
                                    panjang2+0.1), (panjang1+panjang2+0.1))
    ax_sudut1.set_xlim(0, time)
    ax_sudut1.set_ylim(-np.pi, np.pi)
    ax_sudut2.set_xlim(0, time)
    ax_sudut2.set_ylim(-np.pi, np.pi)
    ax_sudutdot1.set_xlim(-np.pi, np.pi)
    ax_sudutdot1.set_ylim(-np.pi, np.pi)
    ax_sudutdot2.set_xlim(-np.pi, np.pi)
    ax_sudutdot2.set_ylim(-np.pi, np.pi)

    return plot_jejak_massa, plot_garis, plot_massa, plot_sudut1, plot_sudut2, plot_sudutdot1, plot_sudutdot2,


def animasi(step):
    global list_jejak_sudut1, list_jejak_sudut2, list_jejak_sudutdot1, list_jejak_sudutdot2, list_jejak, time

    rungekutta4()
    garisXY, massaXY = sudutkeXY()

    list_jejak_sudut1.append(sudut[0, 0])
    list_jejak_sudut2.append(sudut[1, 0])
    list_jejak_sudutdot1.append(sudutdot[0, 0])
    list_jejak_sudutdot2.append(sudutdot[1, 0])
    list_jejak = np.vstack((list_jejak, massaXY))

    plot_massa.set_data(massaXY[:, 0], massaXY[:, 1])
    plot_garis.set_data(garisXY[:, 0], garisXY[:, 1])
    plot_jejak_massa.set_data(list_jejak[:, 0], list_jejak[:, 1])
    plot_sudut1.set_data(
        np.arange(0, len(list_jejak_sudut1)*dt, dt), list_jejak_sudut1)
    plot_sudut2.set_data(
        np.arange(0, len(list_jejak_sudut2)*dt, dt), list_jejak_sudut2)
    plot_sudutdot1.set_data(list_jejak_sudut1, list_jejak_sudutdot1)
    plot_sudutdot2.set_data(list_jejak_sudut2, list_jejak_sudutdot2)
    return plot_jejak_massa, plot_garis, plot_massa, plot_sudut1, plot_sudut2, plot_sudutdot1, plot_sudutdot2,


anim = animation.FuncAnimation(fig, animasi, frames=np.arange(
    0, time, dt), interval=0, init_func=inisialisasi_animasi, blit=True, repeat=False)

plt.show


Using matplotlib backend: Qt5Agg


<function matplotlib.pyplot.show(*, block=None)>

In [ ]:
plt.